# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: Щеголев Олег Борисович

Группа: DS-12

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [1]:
import overpy
import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize
import folium
import numpy as np
from geopy.distance import great_circle as distance
import folium.plugins

def get_country_polygon_by_id(cid: int):
    query = f"""[out:json][timeout:25];
    rel({cid});
    out body;
    >;
    out skel qt; """
    api = overpy.Overpass()
    result = api.query(query)
    linestrings = []
    for way in result.ways:
        coords = []
        for node in way.nodes:
            coords.append((node.lon,node.lat))
        linestrings.append(geometry.LineString(coords))
        borders = unary_union(linemerge([*linestrings]))
    return geometry.MultiPolygon(list(polygonize(borders)))

afgan = get_country_polygon_by_id(303427)


In [167]:

def get_random_point_in_rectangle(a=180, b=360, center=geometry.Point(0,0)):
    coords = np.random.rand(2)
    coords[0] = (coords[0] - 0.5) * a + center.coords[0][0]
    coords[1] = (coords[1] - 0.5) * b + center.coords[0][1]
    if abs(coords[0]) > 90:
        return get_random_point_in_rectangle()
    if coords[1] < -180:
        coords[1] += 360
    elif coords[1] > 180:
        coords[1] -= 360
    return geometry.Point(coords)

def distance_to_polygon(point, polygon=afgan):
    return distance(point.coords, polygon.centroid.coords).km

def inside(point, polygon=afgan):
    return polygon.contains(point)

def search_point_inside_polygon(polygon=afgan):
    random_point = get_random_point_in_rectangle()
    points = [random_point]
    while not inside(random_point):
        dist = distance_to_polygon(random_point)
        new_point = random_point
        while distance_to_polygon(new_point) >= dist:
            new_point = get_random_point_in_rectangle(dist / 110, dist / 110, random_point)
        random_point = new_point
        points.append(random_point)
    points.append(random_point)
    return points

points = search_point_inside_polygon()

In [168]:

m = folium.Map(location = [33.651, 66.357], zoom_start = 1)
folium.GeoJson(afgan).add_to(m)
times = list(np.array(list(range(10, len(points) + 10))) * 1e7)
folium.plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': [
      {
        'type': 'Feature',
        'geometry': {
          'type': 'LineString',
          'coordinates': [point.coords[0] for point in points]
          },
        'properties': {
          'times': times
          }
        }
      ]
    }, auto_play=True,
    loop=False,
    loop_button=True,
    add_last_point=True,
    date_options='ss').add_to(m)
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
# put your code here

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [96]:
from openrouteservice import client
import folium
from shapely.geometry import LineString, Polygon, Point, mapping
from shapely.ops import cascaded_union
import time
from pandas.io.json import json_normalize
import json


p1 = [40.6866, -73.9807]
p2 = [40.7022, -74.0657]
p3 = [40.8837, -73.9364]
p4 = [40.7785, -73.9093] 

bounding_box = Polygon([p1, p2, p3, p4])

with open('rows.json') as json_file:
    data = json.load(json_file)
schools = []
for key in data['data']:
    lat, lon = list(map(float, key[8][7:-1].split()))
    point = Point(lon, lat)
    if bounding_box.contains(point):
        schools.append([lat, lon])
        
def style_function(color):
    return lambda feature: dict(color=color,
                                opacity=0.5,
                                weight=4,)

api_key = '5b3ce3597851110001cf6248efb455d29d6c4eb389c195258e1d56bb'
clnt = client.Client(key=api_key)

map_manhattan = folium.Map( 
                        attr='Map data (c) OpenStreetMap, Tiles (c) <a href="https://heigit.org">GIScience Heidelberg</a>', 
                        location=(40.7583, -73.9788), 
                        zoom_start=12)

popup_route = "<h4>{0} route</h4><hr>" \
             "<strong>Duration: </strong>{1:.1f} mins<br>" \
             "<strong>Distance: </strong>{2:.3f} km" 

coordinates = [[-73.96716, 40.7725], [-73.98954, 40.70316]]
direction_params = {'coordinates': coordinates,
                    'profile': 'driving-car', 
                    'format_out': 'geojson',
                    'preference': 'shortest',
                    'geometry': 'true'}

folium.Marker(list(reversed(coordinates[0])), popup='Central Park entrance from 5th Avenue').add_to(map_manhattan)
folium.Marker(list(reversed(coordinates[1])), popup='Crossroad of Water Street and Washigton Street').add_to(map_manhattan)

avoid_schools = cascaded_union([Point(school).buffer(1.5e-3).simplify(1e-3) for school in schools])
folium.GeoJson(avoid_schools).add_to(map_manhattan)

request = {'coordinates': coordinates, 
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': False,
                 'options': {'avoid_polygons': mapping(avoid_schools)}} 
route = clnt.directions(**request)

distance, duration = route['features'][0]['properties']['summary'].values()
popup = folium.map.Popup(popup_route.format('Route',
                                                 duration/60,
                                                 distance/1000))

folium.GeoJson(route,
               style_function=style_function('black'), 
               name='Route avoiding schools').add_child(popup).add_to(map_manhattan)

map_manhattan.add_child(folium.map.LayerControl())
map_manhattan